In [ ]:
# Import necessary libraries and widgets
!wget -O prbs_test_18_Jun_2025_13_50_02.mat \
https://raw.githubusercontent.com/AlasMac/TCLab-Python-Resources/main/prbs_test_18_Jun_2025_13_50_02.mat

import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from IPython.display import display
import ipywidgets as widgets

# --- ARX simulation function (UNMODIFIED) ---
def ARX_Sim_PID(A, B, k, Kp, Ki, Kd, r, mu, sigma, SIM_TIME, x0, u_thresh):
    """
    Simulates an ARX model with PID control.
    """
    t = np.arange(1, SIM_TIME + 1)  # time vector (1...SIM_TIME)

    # Reproducible Gaussian process noise
    rng = np.random.default_rng(seed=100)
    e = rng.normal(mu, sigma, size=len(t))

    na = len(A)
    nb = len(B)
    L = max(na, nb)

    # Pad A or B so lengths match
    if na > nb:
        B = np.concatenate([B, np.zeros(na - nb)])
        nb = len(B)
    elif nb > na:
        A = np.concatenate([A, np.zeros(nb - na)])
        na = len(A)

    # Preallocate
    ut = np.zeros(len(t) + L)
    y = np.ones(len(t) + L) * x0
    err = np.zeros_like(y)

    integral_err = 0
    alpha = 0.5  # derivative filter coefficient
    dt = 1.0

    # Simulation loop
    for i in range(L + k, SIM_TIME):
        # ARX system dynamics
        y[i] = -np.dot(A[1:], y[i - 1 : i - na : -1]) \
               + np.dot(B, ut[i - k : i - nb - k : -1]) \
               + e[i]

        # Error
        err[i] = r[i] - y[i]

        # Integral error
        integral_err += err[i] * dt

        # PID control law
        d_err = err[i] - err[i - 1]
        ut[i + 1] = (Kp * err[i]
                     + Ki * integral_err
                     + Kd * (alpha * d_err / dt + (1 - alpha) * d_err))

        # Saturation
        ut[i + 1] = np.clip(ut[i + 1], 0, u_thresh)

    # Trim padded part
    y = y[: -L]
    ut = ut[: -L]

    return y, ut, t

# --- Load MATLAB data and System Parameters (UNMODIFIED) ---
filename = 'prbs_test_18_Jun_2025_13_50_02.mat'
data = loadmat(filename)

T = data['T']
U = data['U']

y_data_test = T[0, :]
y_data_test = y_data_test - y_data_test[0]
u_data_test = U[0, :]
t_test = np.arange(len(T))

# --- Model parameters ---
A = [1, -3.88098254, 5.64726905, -3.6515443, 0.88525793]
B = [0.4453368, -1.71110786, 2.49838163, -1.64272524, 0.41012833]
k = 0
sigma = 1e-5

# --- Setpoint ---
N = len(u_data_test)
r = np.zeros(N, dtype=float)
r[100:] = 30 # Step change in setpoint

# --- Plotting Function (SIMPLIFIED for comparison) ---
def plot_sandbox(t, y, u, t2, y2, u2, r, Kp1, Ki1, Kd1, Kp2, Ki2, Kd2):
    """
    Plots the results of the two simulations for comparison.
    """
    plt.figure(figsize=(10, 8))
    plt.suptitle('🌡️ Control Sandbox: Comparing Different PID Parameters', fontsize=14, fontweight='bold')
    
    # Define labels for the legend
    label1 = f'Controller 1: $K_p$={Kp1:.3f}, $K_i$={Ki1:.5f}, $K_d$={Kd1:.3f}'
    label2 = f'Controller 2: $K_p$={Kp2:.3f}, $K_i$={Ki2:.5f}, $K_d$={Kd2:.3f}'

    # Output Plot
    plt.subplot(2, 1, 1)
    plt.plot(t, y, 'b', label=label1)
    plt.plot(t2, y2, 'g', label=label2)
    plt.plot(t, r, '--r', label='Reference')
    plt.ylabel('Output (°C)')
    plt.legend(loc='lower right', fontsize=9)
    plt.grid(True, linestyle=':', alpha=0.6)

    # Control Input Plot
    plt.subplot(2, 1, 2)
    plt.plot(t, u, 'b', label='Control Input 1')
    plt.plot(t2, u2, 'g', label='Control Input 2')
    plt.ylabel('Control Input')
    plt.xlabel('Time (s)')
    plt.legend(loc='lower right')
    plt.grid(True, linestyle=':', alpha=0.6)
    
    plt.tight_layout()
    plt.show()

# --- Main Simulation Function called by the GUI button ---
def run_simulation(b):
    """
    Retrieves widget values, runs the simulations, and plots the results.
    The 'b' argument is required by the button widget.
    """
    # Get gains for Controller 1
    Kp1 = Kp1_widget.value
    Ki1 = Ki1_widget.value
    Kd1 = Kd1_widget.value
    
    # Get gains for Controller 2
    Kp2 = Kp2_widget.value
    Ki2 = Ki2_widget.value
    Kd2 = Kd2_widget.value

    u_thresh = 0.75 # Control Input Saturation Limit

    # Simulate with first set of PID parameters 
    y1, u1, t1 = ARX_Sim_PID(A, B, k, Kp1, Ki1, Kd1, r, 0, sigma, N, y_data_test[0], u_thresh)

    # Simulate with second set of PID parameters
    y2, u2, t2 = ARX_Sim_PID(A, B, k, Kp2, Ki2, Kd2, r, 0, sigma, N, y_data_test[0], u_thresh)

    # Plot both results for comparison
    plot_sandbox(t1, y1, u1, t2, y2, u2, r, Kp1, Ki1, Kd1, Kp2, Ki2, Kd2)

# ----------------- GUI SETUP -----------------

# Define initial gain values
# You can use the last values from your original script as defaults
Kp1_default = 0.1
Ki1_default = 0.0003
Kd1_default = 0.25

Kp2_default = 0.2
Ki2_default = 0.0
Kd2_default = 0.1


# 1. Controller 1 Widgets (Blue)
style1 = {'description_width': 'initial', 'handle_color': 'lightblue'}
Kp1_widget = widgets.FloatText(value=Kp1_default, description='Kp 1:', style=style1)
Ki1_widget = widgets.FloatText(value=Ki1_default, description='Ki 1:', style=style1)
Kd1_widget = widgets.FloatText(value=Kd1_default, description='Kd 1:', style=style1)

# 2. Controller 2 Widgets (Green)
style2 = {'description_width': 'initial', 'handle_color': 'lightgreen'}
Kp2_widget = widgets.FloatText(value=Kp2_default, description='Kp 2:', style=style2)
Ki2_widget = widgets.FloatText(value=Ki2_default, description='Ki 2:', style=style2)
Kd2_widget = widgets.FloatText(value=Kd2_default, description='Kd 2:', style=style2)

# 3. Simulation Button
button = widgets.Button(description="🚀 Run Simulation (Compare)", 
                        button_style='success',
                        layout=widgets.Layout(width='auto', height='40px'))
button.on_click(run_simulation)

# 4. Organizing the Layout
# Use VBox and HBox to arrange the input fields and button
ctrl1_label = widgets.Label(value="🔵 Controller 1 Gains")
ctrl1_box = widgets.VBox([ctrl1_label, Kp1_widget, Ki1_widget, Kd1_widget])

ctrl2_label = widgets.Label(value="🟢 Controller 2 Gains")
ctrl2_box = widgets.VBox([ctrl2_label, Kp2_widget, Ki2_widget, Kd2_widget])

# Combine the two controller input boxes horizontally
input_box = widgets.HBox([ctrl1_box, widgets.VBox([widgets.Label(value=''), button]) , ctrl2_box], 
                         layout=widgets.Layout(justify_content='space-around'))

# Display the GUI
print("Adjust the controller gains below and press the 'Run Simulation' button.")
display(input_box)

'wget' is not recognized as an internal or external command,
operable program or batch file.
